## Load the data and resample
resample the fraud label data, lift the ratio of fraud to non-fraud from 0.1:100 to 1:100

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(1)

In [ ]:
# change it to your own path that places credit card fraud dataset.
#file = "LOCAL_DATA_PATH"
card = pd.read_csv(file)

from math import floor
train_size = floor(card.shape[0]*0.8) #the top "train_size" rows of data will be used as historical data
#used for training model

#the rest will be used as incoming test data.


In [ ]:
test_card = card.iloc[train_size:,:]
test_card_fraud = test_card[test_card['Class']==1]
print('testing data rows: ',test_card.shape[0])
print('testing fraud data rows: ',test_card_fraud.shape[0])

In [ ]:
#upsample fraud data
test_card_fraud_final = pd.concat([test_card_fraud]*80, ignore_index=True)
test_card_final = pd.concat([test_card,test_card_fraud_final], ignore_index=True)
test_card_final = test_card_final.iloc[np.random.RandomState(seed=1).permutation(len(test_card_final))]
print('testing data rows: ',test_card_final.shape[0])
print('testing fraud data rows: ',test_card_fraud_final.shape[0])

## Connect to Firehose

In [ ]:
import time
import base64

class KinesisLoader:
    # TODO: create new stream if stream_name doesn't exist
    
    def __init__(self, client,stream_source,stream_name,batch_size=100,sleep_time = 1):
        """
        The client is a low level boto3 client 
        The stream_source is a dataset used to simulate streaming data, 
            each row contains multiple features of a transaction
        The stream_name here is the name of the kinesis stream.
        The default batch_size here is to match the maximum allowed by Kinesis in a PutRecords request
        """
        self.batch_size = min(batch_size, 100)
        self.kinesis_client = client
        self.stream_source = stream_source
        self.stream_name = stream_name
        self.sleep_time = sleep_time
    #TODO: partition by timestamp 
    def submit_batch(self):
        count = 0
        records_batch = []
        for row in range(self.stream_source.shape[0]):
            '''
            encoding: convert each row to json format, and then to base64 encoded
            '''
            #base64.b64encode(self.stream_source.iloc[0,:].to_json().encode('utf-8'))
            json_row = self.stream_source.iloc[row,:].to_json()
            records_batch.append({'Data':json_row})
            count+=1
            '''
            Request syntax of put_record_batch():
            {
               "DeliveryStreamName": "string",
               "Records": [ 
                    { 
                     "Data": blob
                    }
               ]
            }
            '''
            if count == self.batch_size or row == self.stream_source.shape[0]-1:
                request = {
                'Records':records_batch,
                'DeliveryStreamName':self.stream_name
            }
                #print('just sent out one request: {}'.format(request))
                self.kinesis_client.put_record_batch(**request)
                #self.submit_batch_until_successful(records_batch,response)
                count=0
                records_batch=[]
                time.sleep(self.sleep_time)
                

In [ ]:
import boto3
client = boto3.client( 'firehose')

## Start the stream

In [ ]:
upload_client = KinesisLoader(client,test_card_final.iloc[:1000,:],'trendsmarket',
                              batch_size=10,sleep_time=0.5)
upload_client.submit_batch()